In [ ]:
# parameters
cohort_id = 'AO-LUA_Luanda_colu_2009'

In [ ]:
import yaml
import pandas as pd
import malariagen_data
from pyprojroot import here
import numpy as np
import os

In [ ]:
ag3 = malariagen_data.Ag3(
    # TODO in production build, remove use of simplecache if running inside google cloud
    url="simplecache::gs://vo_agam_release",
    simplecache=dict(cache_storage=(here() / "gcs_cache").as_posix()),
    results_cache=(here() / "malariagen_data_cache").as_posix(),
)
ag3

In [ ]:
# load parameters 
with open(here() / "workflow" / "params.yaml") as params_file:
    params = yaml.safe_load(params_file)
params

In [ ]:
# load window sizes 
outdir = here() / "build" / "h12-calibration"
with open(outdir / f"{cohort_id}.yaml") as yaml:
    window_size_params = yaml.safe_load(params_file)

In [ ]:
# load cohorts to find sample query 
df_cohorts = pd.read_csv(here() / "workflow" / "cohorts.csv").set_index("cohort_id")
cohort = df_cohorts.loc[cohort_id]
cohort

In [ ]:
sample_query = df_cohorts.query("cohort_id in @cohort_id").loc[:, 'sample_query'].to_list()[0]

In [ ]:
if cohort.taxon == 'arabiensis':
    phasing_analysis = 'arab'
else:
    phasing_analysis = 'gamb_colu'
phasing_analysis

In [ ]:
if cohort.cohort_size > params['max_cohort_size']:
    # downsampling for computational efficiency
    cohort_size = params['max_cohort_size']
else:
    # no downsampling
    cohort_size = None 
cohort_size

In [ ]:
for contig in params['ag']['contigs']:
    ag3.h12_gwss(contig=contig, sample_query=sample_query, analysis=phasing_analysis, window_size=window_size_params['h12_window_size'], cohort_size=params['max_cohort_size'])